In [55]:
import os,sys
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

In [5]:
samples_pid_train = pickle.load(open('./data/tree-rep-profiles-partial/TRAIN_pidNames.pkl','rb'))
samples_pid_valid = pickle.load(open('./data/tree-rep-profiles-partial/DEV_pidNames.pkl','rb'))
samples_pid_test = pickle.load(open('./data/tree-rep-profiles-partial/TEST_pidNames.pkl','rb'))

In [25]:
# all many2many pid
all_m2m_pid = []
root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            all_m2m_pid.extend(hl_list)
all_m2m_pid = set(all_m2m_pid)

100%|██████████| 180/180 [00:00<00:00, 3624.21it/s]


* 是否都轉小寫?
* sent2vec hyper parameters
* 

In [161]:
corpus = []
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"]+\ *" #等號、底線被保留

root_dir = './data/tree-rep-profiles_one2many/'
rasmma_dir  = next(os.walk(root_dir))[1]
for dir_ in rasmma_dir:
    fam_dir = next(os.walk(root_dir + dir_))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for hl in hl_list:
                try:
                    if hl in samples_pid_test: #移除在test裡面的hkl
                        hl_list.remove(hl)
                    if hl in samples_pid_valid: #移除在train裡面的hkl
                        hl_list.remove(hl)
                except ValueError:
                    pass
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            for profile in hl_list:
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read().splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line.lower())
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    corpus.append(temp)

# root_dir = './data/tree-rep-profiles_one2one/'
# rasmma_dir  = next(os.walk(root_dir))[1]
# for dir_ in rasmma_dir:
#     fam_dir = next(os.walk(root_dir + dir_))[1]
#     for fam in tqdm(fam_dir):
#         tree_dir = next(os.walk(root_dir + dir_+ '/' + fam))[1]
#         for tree in tree_dir:
#             in_directory = root_dir + dir_ + '/' + fam +  '/' + tree + '/'
#             hl_list = next(os.walk(in_directory))[2]
#             hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
#             for hl in hl_list:
#                 try:
#                     if hl in all_m2m_pid: #移除已經有的pid
#                         hl_list.remove(hl)
#                 except ValueError:
#                     pass
#             hl_list = [os.path.join(in_directory, f) for f in hl_list]
#             hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
#             for profile in hl_list:
#                 with open(profile,encoding='ISO 8859-1') as f:
#                     lines = f.read().splitlines()
#                 for line in lines:
#                     temp = re.sub(dil," ",line)
#                     temp = temp.split(" ")
#                     temp = list(filter(None, temp))
#                     temp = ' '.join(temp)
#                     corpus.append(temp)

with open('data/o2m_Sent2Vec_0521.txt','w') as f:
    f.write('\n'.join(corpus))
corpus[-10:]

100%|██████████| 180/180 [00:06<00:00, 28.52it/s]


['regdeletevalue pr hkus reg software microsoft windows currentversion internet settings pr subk proxyserver ret p',
 'regdeletevalue pr hkus reg software microsoft windows currentversion internet settings pr subk proxyoverride ret p',
 'regdeletevalue pr hkus reg software microsoft windows currentversion internet settings pr subk autoconfigurl ret p',
 'regcreatekey pr hkcc hkey current config pr software microsoft windows currentversion internet settings ret 0',
 'regsetvalue pr hkcc soft ms win internetsettings proxyenable pr reg dword pr 0 ret 0',
 'regcreatekey pr hkus reg pr software microsoft windows currentversion internet settings connections ret 0',
 'regqueryvalue pr hkus reg software microsoft windows currentversion internet settings connections pr subk savedlegacysettings pr 0 pr 0 ret 0',
 'regqueryvalue pr hkus reg software microsoft windows currentversion internet settings connections pr subk savedlegacysettings pr 0 pr 1697e0 ret 0',
 'regcreatekey pr hkus reg pr softw

# Sent2Vec Training time 
* cd ~/Downloads/sent2vec/

## Evaluate

In [178]:
with open('data/o2m_Sent2Vec_0521.txt','r') as f: #asp1_sentences_rep_0116
    corpus = f.read().splitlines()
corpus = sorted(corpus)
corpus[-100:]

['winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret 0 ret n',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest ret p ret 0',
 'winhttpsendrequest

In [179]:
model = sent2vec.Sent2vecModel()
model.load_model('model/sent2vec_o2m_0521.bin')
emb_all = model.embed_sentences(corpus)

In [180]:
api = ""
count = 0
count_all = {}
for row in corpus:
    try:
        count_all[row.split(' ')[0]] += 1
    except KeyError:
        count_all[row.split(' ')[0]] = 1
sorted(count_all.items())

[('a\x7f', 2),
 ('copyfile', 62294),
 ('createfile', 425331),
 ('createprocess', 4723),
 ('createremotethread', 11304),
 ('createthread', 10473),
 ('deletefile', 6754),
 ('exitprocess', 4682),
 ('geturlcacheentryinfo', 25),
 ('httpsendrequest', 1079),
 ('internetconnect', 1121),
 ('internetopen', 1079),
 ('loadlibrary', 238085),
 ('openprocess', 13042),
 ('pã¿\x15ã¤\x10â«qâ\x83ã¨ã¿ã«', 4),
 ('regcreatekey', 99712),
 ('regdeletekey', 708),
 ('regdeletevalue', 5133),
 ('regenumvalue', 162493),
 ('regqueryvalue', 1162572),
 ('regsetvalue', 69863),
 ('terminateprocess', 38),
 ('winexec', 368),
 ('winhttpconnect', 423),
 ('winhttpopen', 426),
 ('winhttpopenrequest', 422),
 ('winhttpreaddata', 392),
 ('winhttpsendrequest', 432),
 ('\x7f', 13),
 ('â\x8deã´pâ\x8deã¸pã¿\x15p', 4),
 ('ã\xa0â\x89eã¼t', 4)]

In [181]:
kk = list(count_all.keys())
for k in api_li:
    if k not in kk:
        print('少了',k)
for k in kk:
    if k not in api_li:
        print('多了',k)

少了 winhttpwritedata
多了 a
多了 pã¿ã¤â«qâã¨ã¿ã«
多了 
多了 âeã´pâeã¸pã¿p
多了 ã âeã¼t


In [182]:
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
'WinHttpWriteData',
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

27

In [183]:
c= 0
df = pd.DataFrame(0, index=api_li, columns=api_li)
index_ = df.index.tolist()
avg = 100
for _ in range(avg):
    c=0
    for k,v in count_all.items():
        pick = random.randrange(c,c+v) #cosine similarity的左邊
        pick = emb_all[pick,:] #sorted corpus的embedding_all
        c1 = 0
        try:
            row = index_.index(k)
        except ValueError:
            continue
        for k1,v1 in count_all.items(): # cosine similarity的右邊
            pick1 = random.randrange(c1,c1+v1)
            pick1 = emb_all[pick1,:]
            score = cosine_similarity([pick],[pick1]) #cosine_similarity
    #         print(score)
            c1 = c1+v1
            try:
                col = index_.index(k1)
                df.iloc[row,col] += float(score)
            except ValueError:
                continue
        c = c+v
df = df/avg #similarity
# df = (df-np.min(df.values))/(np.max(df.values)-np.min(df.values)) #distance normalize
df

,loadlibrary,createprocess,openprocess,exitprocess,terminateprocess,winexec,createremotethread,createthread,copyfile,createfile,...,winhttpconnect,winhttpopen,winhttpopenrequest,winhttpreaddata,winhttpsendrequest,winhttpwritedata,internetopen,internetconnect,httpsendrequest,geturlcacheentryinfo
loadlibrary,0.690161,0.247582,0.386947,0.221414,0.149436,0.301718,0.047488,0.198012,0.251804,0.104924,...,0.231114,0.119964,0.270295,0.245907,0.124497,0.0,0.269621,0.332581,0.034388,-0.118206
createprocess,0.222038,0.876652,0.431557,0.414545,0.495939,0.867542,0.480651,0.459646,0.654046,0.539904,...,0.430373,0.489291,0.503856,0.347342,0.239362,0.0,0.470942,0.303222,0.264047,0.316302
openprocess,0.378618,0.446242,0.945792,0.448235,0.446730,0.519514,0.493350,0.484413,0.340723,0.384605,...,0.428220,0.372048,0.469592,0.423391,0.297018,0.0,0.505641,0.405273,0.295238,0.170687
exitprocess,0.237695,0.418011,0.451307,0.854074,0.551825,0.489413,0.458055,0.527580,0.311845,0.322891,...,0.442053,0.506402,0.532011,0.242762,0.078632,0.0,0.540131,0.528421,0.048288,0.131230
terminateprocess,0.123774,0.450920,0.430603,0.504522,0.643378,0.512740,0.588923,0.603342,0.433965,0.456257,...,0.538384,0.468274,0.473678,0.372714,0.205053,0.0,0.498970,0.388412,0.356781,0.387118
winexec,0.307179,0.869479,0.524640,0.497597,0.565551,0.992135,0.485187,0.476391,0.532113,0.486759,...,0.493062,0.491048,0.653181,0.416711,0.290153,0.0,0.541821,0.470771,0.310230,0.304897
createremotethread,0.002152,0.486473,0.503048,0.456057,0.639787,0.486053,0.948561,0.794258,0.547573,0.656953,...,0.593050,0.517941,0.418456,0.288028,0.140480,0.0,0.514110,0.267561,0.330779,0.402093
createthread,0.195602,0.438374,0.499622,0.532707,0.642083,0.477271,0.787613,0.919359,0.538815,0.601905,...,0.524555,0.597168,0.456227,0.354928,0.197303,0.0,0.612864,0.348309,0.247817,0.299132
copyfile,0.234409,0.658167,0.352270,0.327719,0.433506,0.548036,0.534816,0.533938,0.910048,0.641976,...,0.426188,0.419994,0.320643,0.292883,0.212798,0.0,0.430256,0.159185,0.199643,0.190638
createfile,0.087129,0.546444,0.390232,0.319475,0.452870,0.483674,0.651962,0.605427,0.641453,0.872701,...,0.441005,0.592982,0.387292,0.227468,0.177363,0.0,0.472420,0.084525,0.246700,0.280306


In [184]:
lib = ['LoadLibrary']
proc = ['CreateProcess', 'OpenProcess', 'ExitProcess', 'TerminateProcess', 'WinExec',
        'CreateRemoteThread', 'CreateThread']
file = ['CopyFile', 'CreateFile', 'DeleteFile']
reg = ['RegSetValue', 'RegCreateKey', 'RegDeleteKey', 'RegDeleteValue',
       'RegQueryValue', 'RegEnumValue']
net = ['WinHttpConnect', 'WinHttpOpen', 'WinHttpOpenRequest', 'WinHttpReadData', 'WinHttpSendRequest',
       'WinHttpWriteData', 'InternetOpen', 'InternetConnect', 'HttpSendRequest', 'GetUrlCacheEntryInfo']

#lower?
lib = [x.lower() for x in lib]
proc = [x.lower() for x in proc]
file = [x.lower() for x in file]
reg =[x.lower() for x in reg]
net = [x.lower() for x in net]

# df.loc['']
x = [lib,proc,file,reg,net]
x = sum(x,[])
index_ , len(index_)

(['loadlibrary',
  'createprocess',
  'openprocess',
  'exitprocess',
  'terminateprocess',
  'winexec',
  'createremotethread',
  'createthread',
  'copyfile',
  'createfile',
  'deletefile',
  'regsetvalue',
  'regcreatekey',
  'regdeletekey',
  'regdeletevalue',
  'regqueryvalue',
  'regenumvalue',
  'winhttpconnect',
  'winhttpopen',
  'winhttpopenrequest',
  'winhttpreaddata',
  'winhttpsendrequest',
  'winhttpwritedata',
  'internetopen',
  'internetconnect',
  'httpsendrequest',
  'geturlcacheentryinfo'],
 27)

In [185]:
up=0
down = len(index_)
down_cat = len(x)
up_cat = 0
up_cat2 = 0
lib_lib=lib_proc=lib_file=lib_reg=lib_net = 0
proc_lib=proc_proc=proc_file=proc_reg=proc_net = 0
file_lib=file_proc=file_file=file_reg=file_net = 0
reg_lib=reg_proc=reg_file=reg_reg=reg_net = 0
net_lib=net_proc=net_file=net_reg=net_net = 0
for api in index_:
    if df.loc[api].idxmax() == api:
        up+=1
    else:
        print(api)
    baseline_lib = df.loc[api,lib].mean()
    baseline_proc =  df.loc[api,proc].mean()
    baseline_file = df.loc[api,file].mean()
    baseline_reg = df.loc[api,reg].mean()
    baseline_net = df.loc[api,net].mean()
    max_value = np.max([baseline_lib,baseline_proc,baseline_file,baseline_reg,baseline_net])
    if (api in lib):
        lib_lib += baseline_lib
        lib_proc += baseline_proc
        lib_file += baseline_file
        lib_reg += baseline_reg
        lib_net += baseline_net
        if (max_value==baseline_lib):
            up_cat+=1
    elif api in proc :
        proc_lib += baseline_lib
        proc_proc += baseline_proc
        proc_file += baseline_file
        proc_reg += baseline_reg
        proc_net += baseline_net
        if max_value==baseline_proc:
            up_cat+=1
    elif api in file :
        file_lib += baseline_lib
        file_proc += baseline_proc
        file_file += baseline_file
        file_reg += baseline_reg
        file_net += baseline_net
        if max_value==baseline_file:
            up_cat+=1    
    elif api in reg:
        reg_lib += baseline_lib
        reg_proc += baseline_proc
        reg_file += baseline_file
        reg_reg += baseline_reg
        reg_net += baseline_net
        if max_value==baseline_reg:
            up_cat+=1    
    elif api in net:
        net_lib += baseline_lib
        net_proc += baseline_proc
        net_file += baseline_file
        net_reg += baseline_reg
        net_net += baseline_net
        if max_value==baseline_net:
            up_cat+=1
if np.max([lib_lib,lib_proc,lib_file,lib_reg,lib_net]) == lib_lib:
    up_cat2+=1
if np.max([proc_lib,proc_proc,proc_file,proc_reg,proc_net]) == proc_proc:
    up_cat2+=1
if np.max([file_lib,file_proc,file_file,file_reg,file_net]) == file_file:
    up_cat2+=1
if np.max([reg_lib,reg_proc,reg_file,reg_reg,reg_net]) == reg_reg:
    up_cat2+=1
if np.max([net_lib,net_proc,net_file,net_reg,net_net]) == net_net:
    up_cat2+=1
        
print('自己還原自己-ACC:',(up/down)*100,'%') #self
print('自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的):',(up_cat/down_cat)*100,'%') #micro
print('自己那類屬於自己那一類-ACC:',(up_cat2/5)*100,'%') #macro

regqueryvalue
winhttpwritedata
自己還原自己-ACC: 92.5925925925926 %
自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 85.18518518518519 %
自己那類屬於自己那一類-ACC: 80.0 %


:~/Downloads/sent2vec$ ./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2m_o2o_Sent2Vec_0521.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2m_o2o_0521 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 13 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2
* 自己還原自己-ACC: 88.88888888888889 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的) 81.48148148148148 %
* 自己那類屬於自己那一類-ACC 100.0 %
* 小寫的更高
***
濾掉=號、底線、轉小寫:
regqueryvalue
winhttpwritedata
* 自己還原自己-ACC: 92.5925925925926 %
* 自己屬於自己那一類-ACC(他自己那類的分數平均是不是所有類別分數最高的): 85.18518518518519 %
* 自己那類屬於自己那一類-ACC: 80.0 %
./fasttext sent2vec -input ~/leoqaz12_python/ANTS/seqgen_famclf/data/o2m_Sent2Vec_0521.txt -output ~/leoqaz12_python/ANTS/seqgen_famclf/model/sent2vec_o2m_0521 -minCount 0 -dim 700 -epoch 15 -lr 0.3 -wordNgrams 18 -loss ns -neg 10 -thread 35 -dropoutK 1 -minCountLabel 0 -bucket 2000000 -t 0.00001 -numCheckPoints 2

## 